In [48]:
!git pull

remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/basilwong/awstest1
   66882a7..324af85  master     -> origin/master
Updating 66882a7..324af85
Fast-forward
 source-separation-input/drake-toosie_slide.mp3 | Bin 5933485 -> 0 bytes
 source-separation-input/input1.mp3             | Bin 0 -> 480906 bytes
 source-separation-input/input2.mp3             | Bin 0 -> 480906 bytes
 source-separation-input/input3.mp3             | Bin 0 -> 480906 bytes
 source-separation-input/input4.mp3             | Bin 0 -> 480906 bytes
 source-separation-input/input5.mp3             | Bin 0 -> 480906 bytes
 source-separation-input/input6.mp3             | Bin 0 -> 480906 bytes
 source-separation-input/input7.mp3             | Bin 0 -> 480906 bytes
 source-separation-input/input8.mp3             | Bin 0 -> 480906 bytes
 source-separation-input/input9.mp3     

### Add Dependencies

In [15]:
import sagemaker as sage
from sagemaker import get_execution_role

import zipfile
import os

from sagemaker import ModelPackage


# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.append('src')

!pip install pydub

import audio_util

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [16]:
# Execution role
role = get_execution_role()
# S3 prefixes
common_prefix = "source_separation"
batch_inference_input_prefix = common_prefix + "/batch-inference-input-data"
# Sagemaker Session
sagemaker_session = sage.Session()
# Arn for Source Separator Model Package
modelpackage_arn = 'arn:aws:sagemaker:us-east-2:057799348421:model-package/source-separation-v11570291536-75ed8128ecee95e142ec4404d884ecad'



### Creating the Model

In [51]:
from sagemaker import ModelPackage

def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session, content_type='application/x-recordio-protobuf')

model = ModelPackage(role=role,
                     model_package_arn=modelpackage_arn,
                     sagemaker_session=sagemaker_session,
                     predictor_cls=predict_wrapper)

### Running the Batch Job

Note that if the initial audio file is longer than around 30 seconds, it is too large for the model. The split_mp3() method in  src.audio_util works around this by splitting an mp3 file into 30 second segments. 

In [52]:
batch_input_folder = "source-separation-input"


transform_input = sagemaker_session.upload_data(batch_input_folder, key_prefix=batch_inference_input_prefix)
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-075178354542/source_separation/batch-inference-input-data


In [53]:
import json 
import uuid

bucket = sagemaker_session.default_bucket()

transformer = model.transformer(1, 'ml.m4.xlarge', strategy='SingleRecord', output_path='s3://'+bucket+'/'+common_prefix+'/batch-transform-output')
transformer.transform(transform_input, content_type='application/x-recordio-protobuf')
transformer.wait()

print("Batch Transform output saved to " + transformer.output_path)

....................Starting the inference server with 4 workers.
[2020-04-12 10:41:46 +0000] [12] [INFO] Starting gunicorn 19.9.0
[2020-04-12 10:41:46 +0000] [12] [INFO] Listening at: unix:/tmp/gunicorn.sock (12)
[2020-04-12 10:41:46 +0000] [12] [INFO] Using worker: gevent
[2020-04-12 10:41:46 +0000] [16] [INFO] Booting worker with pid: 16
[2020-04-12 10:41:47 +0000] [17] [INFO] Booting worker with pid: 17
[2020-04-12 10:41:47 +0000] [18] [INFO] Booting worker with pid: 18
[2020-04-12 10:41:47 +0000] [19] [INFO] Booting worker with pid: 19
Testing...
Testing...
2020-04-12 10:42:08.875699: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
169.254.255.130 - - [12/Apr/2020:10:42:09 +0000] "GET /ping HTTP/1.1" 200 1 "-" "Go-http-client/1.1"
169.254.255.130 - - [12/Apr/2020:10:42:09 +0000] "GET /execution-parameters HTTP/1.1" 404 2 "-" "Go-http-client/1.1"
Input path : /tmp/audio_file_1586688129

169.254.255.130 - - [12/Apr/2020:10:48:30 +0000] "POST /invocations HTTP/1.1" 200 4379043 "-" "Go-http-client/1.1"
169.254.255.130 - - [12/Apr/2020:10:48:30 +0000] "POST /invocations HTTP/1.1" 200 4379043 "-" "Go-http-client/1.1"

169.254.255.130 - - [12/Apr/2020:10:48:30 +0000] "POST /invocations HTTP/1.1" 200 4379043 "-" "Go-http-client/1.1"
169.254.255.130 - - [12/Apr/2020:10:48:30 +0000] "POST /invocations HTTP/1.1" 200 4379043 "-" "Go-http-client/1.1"
Batch Transform output saved to s3://sagemaker-us-east-2-075178354542/source_separation/batch-transform-output


### Processing the Batch Output

In [59]:
import boto3
# Downloading files from s3.
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(sagemaker_session.default_bucket())
prefix = "source_separation/batch-transform-output/"
i = 0
audio_util.clear_folder('source-separation-output/batch-transform-output')
for object_summary in my_bucket.objects.filter(Prefix=prefix):
    i = i + 1
    file_name = object_summary.key.split('/')[-1]
    print(file_name)
    my_bucket.download_file(prefix+ file_name, 'source-separation-output/batch-transform-output/output-{}.zip'.format(i))

input1.mp3.out
input2.mp3.out
input3.mp3.out
input4.mp3.out
input5.mp3.out
input6.mp3.out
input7.mp3.out
input8.mp3.out
input9.mp3.out


In [64]:
# Extracting files from zip files. 
audio_util.clear_folder('source-separation-output/extracted')
for file in os.listdir('source-separation-output/batch-transform-output'):
    print(file)
    with zipfile.ZipFile('source-separation-output/batch-transform-output/'+file, 'r') as zip_ref:
        zip_ref.extractall('source-separation-output/extracted/'+file.split('.')[0]+'/')

output-3.zip
output-6.zip
output-8.zip
output-9.zip
output-1.zip
output-4.zip
output-7.zip
output-2.zip
output-5.zip


In [65]:
import os
# Separating the vocal files and the background sound files.
audio_util.clear_folder('source-separation-output/vocals')
audio_util.clear_folder('source-separation-output/background')
for i, folder in enumerate(os.listdir('source-separation-output/extracted/')):
    for file in os.listdir('source-separation-output/extracted/' + folder + '/output'):
        print(str(i) + file)
        if "vocals" in file:
            os.rename('source-separation-output/extracted/' + folder + '/output/' + file, 'source-separation-output/vocals/' + str(i) + file)
        elif "accompaniment" in file:
            os.rename('source-separation-output/extracted/' + folder + '/output/' + file, 'source-separation-output/background/' + str(i) + file)

0audio_file_1586688313.4434075.mp3_accompaniment.wav
0audio_file_1586688313.4434075.mp3_vocals.wav
1audio_file_1586688359.6707158.mp3_vocals.wav
1audio_file_1586688359.6707158.mp3_accompaniment.wav
2audio_file_1586688451.638458.mp3_accompaniment.wav
2audio_file_1586688451.638458.mp3_vocals.wav
3audio_file_1586688129.2001228.mp3_vocals.wav
3audio_file_1586688129.2001228.mp3_accompaniment.wav
4audio_file_1586688267.341563.mp3_vocals.wav
4audio_file_1586688267.341563.mp3_accompaniment.wav
5audio_file_1586688221.484059.mp3_accompaniment.wav
5audio_file_1586688221.484059.mp3_vocals.wav
6audio_file_1586688175.3730624.mp3_accompaniment.wav
6audio_file_1586688175.3730624.mp3_vocals.wav
7audio_file_1586688497.7542431.mp3_vocals.wav
7audio_file_1586688497.7542431.mp3_accompaniment.wav
8audio_file_1586688405.760855.mp3_accompaniment.wav
8audio_file_1586688405.760855.mp3_vocals.wav


### Transcribe the Vocal Files

In [17]:
# Upload the Vocal files onto s3
local_vocals_folder = "source-separation-output/vocals/"
transcribe_input_prefix = "transcribe-input"

transcribe_input = sagemaker_session.upload_data(local_vocals_folder, key_prefix=transcribe_input_prefix)
print("Transcribe input uploaded to " + transcribe_input)

Transcribe input uploaded to s3://sagemaker-us-east-2-075178354542/transcribe-input


In [19]:
import boto3
file = "0audio_file_1586688313.4434075.mp3_vocals.wav"

transcribe = boto3.client('transcribe')
uri_prefix = "https://sagemaker-us-east-2-075178354542.s3.us-east-2.amazonaws.com/transcribe-input/"
job_uri = uri_prefix + file
job_name = "tester1"
transcribe.start_transcription_job(
    TranscriptionJobName=job_name,
    Media={'MediaFileUri': job_uri},
    MediaFormat='wav',
    LanguageCode='en-US',
)
while True:
    status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break

print(status)

{'TranscriptionJob': {'TranscriptionJobName': 'tester1', 'TranscriptionJobStatus': 'COMPLETED', 'LanguageCode': 'en-US', 'MediaSampleRateHertz': 44100, 'MediaFormat': 'wav', 'Media': {'MediaFileUri': 'https://sagemaker-us-east-2-075178354542.s3.us-east-2.amazonaws.com/transcribe-input/0audio_file_1586688313.4434075.mp3_vocals.wav'}, 'Transcript': {'TranscriptFileUri': 'https://s3.us-east-2.amazonaws.com/aws-transcribe-us-east-2-prod/075178354542/tester1/167826fc-0a6b-40a5-9d80-d555bf8e2cc8/asrOutput.json?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEDsaCXVzLWVhc3QtMiJGMEQCIHtUTq6DM8hxm%2BWULGwq01NjfRZBXZPXPkHU1Nh6jS98AiA8paqPgQZknQspD59ZOn4TIMojWkhaP%2Bqdzc6esbdPJSq0AwhUEAEaDDI0NjM2MTMyMjc1MiIM%2B8Pbi5mxWpiplhFqKpEDgtAijENS3UMjKYNsq25jkr9oyae7aI19UgGWtVISQF8SB08n%2Boi8mkGPVjeMI87GxZqRnoVqrwZE12bMKm4M%2B0tuczNvD2daSmPgJU6BBcpNdGLZ74Cl59%2BOrk4boS0Z9oydaOtTo0oF%2Ff72jm7wpCP%2FYw82o%2BL7fMncdkDfSJ%2Bzp%2FReyeiaF48MAaXxB9Z8zyo0UIdecB686Cs6ARIo9c6AoHwo1BDUZm9oEACQ2%2BMRYXp%2BTdqSSEz6m%2F9VHhC6cjT8O

In [20]:
import requests
r = requests.get(url=status['TranscriptionJob']['Transcript']['TranscriptFileUri'])
print(r.json())
transcribe.delete_transcription_job(TranscriptionJobName="tester1")

{'jobName': 'tester1', 'accountId': '075178354542', 'results': {'transcripts': [{'transcript': "that foot right foot. Basically, I'm saying Either way we about so I can let this one slide to see Slide. Then I hated double time and I hit a Schmidt because we spent a block a couple times. If it's not the right time, I will always be another time. I'm not even tripping with him in a sometime. You can't describe the pressure of you putting know myself, really. I just can't afford a little nobody get a move on. Shaky with."}], 'items': [{'start_time': '0.52', 'end_time': '0.88', 'alternatives': [{'confidence': '0.9907', 'content': 'that'}], 'type': 'pronunciation'}, {'start_time': '0.89', 'end_time': '1.31', 'alternatives': [{'confidence': '0.9762', 'content': 'foot'}], 'type': 'pronunciation'}, {'start_time': '1.88', 'end_time': '2.4', 'alternatives': [{'confidence': '1.0', 'content': 'right'}], 'type': 'pronunciation'}, {'start_time': '2.41', 'end_time': '2.78', 'alternatives': [{'confide

{'ResponseMetadata': {'RequestId': '381b2593-63f9-440e-8360-7b0c32a2c9a7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 13 Apr 2020 03:27:35 GMT',
   'x-amzn-requestid': '381b2593-63f9-440e-8360-7b0c32a2c9a7',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [13]:
import boto3
from datetime import datetime

transcribe = boto3.client('transcribe')
output_bucket_name = "transcribe-output"
uri_prefix = "https://sagemaker-us-east-2-075178354542.s3.us-east-2.amazonaws.com/transcribe-input/"
finshed_jobs = dict()

for file in os.listdir(local_vocals_folder):

    print("Transcribing: " + file)
    print("Job: " + str(i))
    job_uri = uri_prefix + file
    transcribe.start_transcription_job(
        TranscriptionJobName=file,
        Media={'MediaFileUri': job_uri},
        MediaFormat='wav',
        LanguageCode='en-US',
    )
    while True:
        status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
        if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
            break
    
    finished_jobs[file] = status

NameError: name 'local_vocals_folder' is not defined

In [87]:

import boto3
# Downloading files from s3.
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(sagemaker_session.default_bucket())
prefix = "source_separation/batch-transform-output/"
i = 0
audio_util.clear_folder('source-separation-output/batch-transform-output')
for object_summary in my_bucket.objects.filter(Prefix=prefix):
    i = i + 1
    file_name = object_summary.key.split('/')[-1]
    print(file_name)
    my_bucket.download_file(prefix+ file_name, 'source-separation-output/batch-transform-output/output-{}.zip'.format(i))

BadRequestException: An error occurred (BadRequestException) when calling the DeleteTranscriptionJob operation: The requested job couldn't be found. Check the job name and try your request again.

In [ ]:
def load_json(file):
    """Load in JSON file and return as dict"""

    json_filepath = Path(file)
    assert json_filepath.is_file(), "JSON file does not exist"

    data = json.load(open(json_filepath.absolute(), "r", encoding="utf-8"))
    assert "jobName" in data
    assert "results" in data
    assert "status" in data

    assert data["status"] == "COMPLETED", "JSON file not shown as completed."

    return data